
# Generating Bash Code with Granite Code and Ollama


> **NOTE:** This recipe assumes you are working on a Linux, MacOS, or other UNIX-compatible system. While we haven't tested on Windows, some of the examples may generate valid DOS or PowerShell output.  

### Prerequisite: Install Ollama and Granite Code models

1. [Download and Start Ollama](https://ollama.com/download)
1. Install Granite Code 20b: `ollama pull granite-code:20b`
1. Install Granite Code 8b: `ollama pull granite-code:8b`

In [ ]:
!pip install ollama


## One-shot Prompt with Granite Code 20b

In One-shot prompting, you provide the model with a question and no examples. The model will generate an answer given its training. Larger models tend to do better at this task.

Use the [ollama-python package](https://github.com/ollama/ollama-python) to access the model.

In [ ]:
import ollama

Let's write two helper functions that we'll use for all our queries. First, we'll find it useful to determine the name of our operating system and use that string in queries. This is because shell commands sometimes have different options on Linux vs. MacOS, etc. We'll write our queries so they take this difference into account. 

> **TIP:** If you are using MacOS, you can install Linux-compatible versions of many commands. Consider these two options: 
> 
> * Install GNU Coreutils on a Mac. See [these instructions](https://superuser.com/questions/476575/replace-os-xs-shell-commands-with-the-linux-versions).
> * Install [HomeBrew](https://brew.sh/) and use it to install Linux-compatible (and other) tools.

In [ ]:
import platform

def os_name():
    os_name = platform.system()
    # It turns out, using "MacOS" is better than "Darwin", which is what gets returned on MacOS. 
    # For all other cases, the returned value should be fine as is, so we map the result to the desired
    # name, but only for MacOS...
    name_map = {'Darwin': 'MacOS'}
    # ... then pass the os_name value as the second arg, which is used as the default return value.
    return name_map.get(os_name, os_name)  

In [ ]:
print(f"My OS is {os_name()}")

Now let's write a helper function for running queries, wrapping the Ollama `generate()` API call. The user specifies the prompt and a model name, but we define the default model name for Granite Code 20B, `granite-code:20b`. Also, note how we add additional context to the user's input prompt, such as _"make sure you write code that works for _my_ system!"_ (We'll see another way to do this below.)

The reason we print the result, then return it, is to get the best output.

In [ ]:
def query(prompt: str, model: str = 'granite-code:20b') -> str:
    response = ollama.generate(
        model=model, 
        prompt=f"{prompt}. Make sure you generate code that is {os_name()}-compatible!")
    result = response["response"]
    print(result)
    return result

In [ ]:
result1 = query("""
    Show me a bash script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time 
    for each file in the output.""")

Paste the command in the next cell. _**Keep the `!` shown**_, e.g., `!ls -al`, so Jupyter knows to run the command as a shell script instead of Python. (You can omit lines like `#!/bin/bash`.) 

Does the script work? If not try running the query again. Also try modifying the query string. What difference do these steps make?

In [ ]:
!

We explore execution of generated shell code in the next recipe we recommend you study after this one, [../Text_to_Shell_Exec](../Text_to_Shell_Exec/Text_to_Shell_Exec.ipynb).

## Few-shot Prompting with Granite Code 8b

In few-shot prompting, you provide the model with a question and some examples. The model will generate an answer given its training. The additional examples help the model zero in on a pattern, which may be required for smaller models to perform well at this task.

In [ ]:
examples = """
Question:
Recursively find files that match '*.js', and filter out files with 'excludeddir' in their paths.
Answer:
find . -name '*.js' | grep -v excludeddir

Question:
Dump \"a0b\" as hexadecimal bytes
Answer:
printf \"a0b\" | od -tx1

Question:
create a tar ball of all pdf files in the current folder and any subdirectories.
Answer:
find . -name '*.pdf' | xargs tar czvf pdf.tar

Question:
Sort all files and directories in the current directory, but no subdirectories, according to modification time, and print only the seven most recently modified items
Answer:
find . -maxdepth 1 -exec stat -f "%Sm {}" \; | sort -n -r | tail -n 7

Question:
find all the empty directories in and under the current directory.
Answer:
find . -type d -empty

"""

Let's define another helper function for calling `ollama.chat()`. Why it is called `chat1()` will be explained below.

In [ ]:
def chat1(prompt: str, examples: str = examples, model: str ='granite-code:8b') -> str:
    user_prompt = f"""
        {examples}
        Question:
        {prompt}. Make sure you generate code that is {os_name()}-compatible!
        Answer:"""
    response = ollama.chat(model=model, messages=[
      {
        'role': 'user',
        'content': user_prompt
      },
    ])
    result = response['message']['content']
    print(result)
    return result

In [ ]:
result2 = chat1("""
    Show me a bash script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time 
    for each file in the output.""")

## Adding a System Prompt

Finally, a _system prompt_ is the preferred way to provide additional instructions and clarity about the context for a task, especially when this same information applies for _all_ user queries in the application. When you are building an AI-enabled application for a set of use cases, you will probably spend a lot of time refining the system prompt to maximize the quality of the results!

Here we define a `default_system_prompt` to let the model know what we expect from it.

So, let's define a final helper function, `chat()`, that includes a system prompt, where `default_system_prompt` is the default. Also, note that we move the sentence `Make sure you only generate code that is {os_name()}-compatible!` to the system prompt, where it really belongs!

In [ ]:
default_system_prompt = f"""
    You are a helpful software engineer. You write clear, concise, 
    well-commented code. Make sure you only generate code that is 
    {os_name()}-compatible!
    """

def chat(prompt: str, 
         system_prompt:str = default_system_prompt,
         examples: str = examples, 
         model: str ='granite-code:8b') -> str:
    user_prompt = f"""
        {examples}
        Question:
        {prompt}
        Answer:""" 
    response = ollama.chat(model=model, messages=[
      {
        'role':'system',
        'content': system_prompt  
      },
      {
        'role': 'user',
        'content': user_prompt
      },
    ])
    result = response['message']['content']
    print(result)
    return result

In [ ]:
result3 = chat("""
    Show me a bash script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time 
    for each file in the output.""")

If you modify `chat()` to return the whole `response`, what additional information do you get?